In [1]:
!pip install google-play-scraper

In [1]:
import json
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [2]:
app_packages = ['com.miHoYo.GenshinImpact']

len(app_packages)


1

In [3]:
app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang='en', country='us')
  del info['comments']
  app_infos.append(info)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  3.72it/s]


In [4]:
def print_json(json_object):
  json_str = json.dumps(
    json_object,
    indent=2,
    sort_keys=True,
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [5]:
print_json(app_infos[0])

{
  "adSupported": false,
  "appId": "com.miHoYo.GenshinImpact",
  "containsAds": false,
  "contentRating": "Teen",
  "contentRatingDescription": "Alcohol Reference, Fantasy Violence",
  "currency": "USD",
  "description": "Step into Teyvat, a vast world teeming with life and flowing with elemental energy.\r\n\r\nYou and your sibling arrived here from another world. Separated by an unknown god, stripped of your powers, and cast into a deep slumber, you now awake to a world very different from when you first arrived.\r\n\r\nThus begins your journey across Teyvat to seek answers from The Seven \u2014 the gods of each element. Along the way, prepare to explore every inch of this wondrous world, join forces with a diverse range of characters, and unravel the countless mysteries that Teyvat holds...\r\n\r\nMASSIVE OPEN WORLD\r\n\r\nClimb any mountain, swim across any river, and glide over the world below, taking in the jaw-dropping scenery each step of the way. And if you stop to investigat

In [6]:
def format_title(title):
  sep_index = title.find(':') if title.find(':') != -1 else title.find('-')
  if sep_index != -1:
    title = title[:sep_index]
  return title[:10]

#fig, axs = plt.subplots(2, len(app_infos) // 2, figsize=(14, 5))

In [7]:
app_infos_df = pd.DataFrame(app_infos)
app_infos_df.to_csv('apps.csv', index=None, header=True)

In [8]:
app_reviews = []

for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 200 if score == 3 else 100,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.19s/it]


In [9]:
print_json(app_reviews[0])

{
  "appId": "com.miHoYo.GenshinImpact",
  "at": "2022-10-10 09:37:48",
  "content": "When I got finished with the verifying file and preparing load data stuff I got towards I think the 1st scene, I couldn't see the images of the scene except it was all white and the audio was pretty terrible it sounded scratchy, I checked my earbuds if they are the problem but they aren't so maybe it's because i am on a laptop(Lenovo Chromebook)? I don't know Edit: I tried maybe switching servers would work... it still had the same problems.It is an Interesting game but after this I deleted it",
  "repliedAt": "2022-10-10 17:36:17",
  "replyContent": "We sincerely apologize for this issue, Traveler. Please send us an email or contact Customer Service via in-game feedback for specific help with this issue.",
  "reviewCreatedVersion": "3.1.0_10443587_10457664",
  "reviewId": "73da55bb-9634-4991-9b7e-056d0f38818e",
  "score": 1,
  "sortOrder": "most_relevant",
  "thumbsUpCount": 20,
  "userImage": "https

In [10]:
len(app_reviews)

1200

In [11]:
app_reviews_df = pd.DataFrame(app_reviews)
app_reviews_df.to_csv('reviewsOct29.csv', index=None, header=True)